In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from vae_earlystopping import EarlyStopping
from torch.utils.data import  DataLoader, Dataset
from model_vae import CVAE,cvae_loss_each,cvae_loss_optimized,cvae_loss
import pandas as pd
import matplotlib.pyplot as plt

x_data = np.load('./data/metal.npy',allow_pickle = True)
c_data = np.load('./data/reaction.npy',allow_pickle = True)
sup_data = np.load('./data/support.npy',allow_pickle=True)


In [2]:
from sklearn.model_selection import train_test_split
x_train,x_test,c_train,c_test = train_test_split(x_data,c_data,test_size= 0.4, shuffle=False)
x_val,x_test,c_val,c_test = train_test_split(x_test,c_test,test_size = 0.5, shuffle=False)
print(x_train.shape,x_val.shape,x_test.shape)

(3544, 24) (1182, 24) (1182, 24)


In [6]:
from sklearn.preprocessing import StandardScaler
x_scaler = StandardScaler()
c_scaler = StandardScaler()
# x정규화
x_train = x_scaler.fit_transform(x_train)
x_test = x_scaler.fit_transform(x_test)
x_val = x_scaler.fit_transform(x_val)
# c정규화
c_train = c_scaler.fit_transform(c_train)
c_val = c_scaler.transform(c_val)
c_test = c_scaler.transform(c_test)

In [8]:
x_train = torch.tensor(x_train, dtype=torch.float32)
x_val   = torch.tensor(x_val,   dtype=torch.float32)
x_test  = torch.tensor(x_test,  dtype=torch.float32)

c_train = torch.tensor(c_train, dtype=torch.float32)
c_val   = torch.tensor(c_val,   dtype=torch.float32)
c_test  = torch.tensor(c_test,  dtype=torch.float32)

In [ ]:
x_dim = x_train.shape[1]
c_dim = c_train.shape[1]
z_dim = 16
x_hat, mu, logvar = model(x_train,c_train)
loss, recon,kl = cvae_loss(x_hat,x_train,mu,logvar,beta = 0.01)

In [21]:
from torch.utils.data import TensorDataset,DataLoader
batch_size = 64
learning_rate = 1e-3
z_dim = 16
epochs = 200
train_dataset = TensorDataset(x_train,c_train)
val_dataset = TensorDataset(x_val,c_val)
train_loader = DataLoader(train_dataset,batch_size=batch_size,shuffle= True)
val_loader = DataLoader(val_dataset,batch_size = batch_size, shuffle = False)

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CVAE(x_dim = x_dim,c_dim=c_dim,z_dim = z_dim).to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [24]:
## x_test,c_test가 torch tensor로 만들어진 뒤에 실행
test_loader = DataLoader(
    TensorDataset(x_test,c_test),
    batch_size = 128,
    shuffle = False
)
train_losses = []
val_losses = []

early = EarlyStopping(patience = 30, min_delta=1e-4)
for epoch in range(1,epochs+1):
    model.train()
    train_total_loss = 0.0
    #batch로 실행하기
    for batch_x,batch_c in train_loader:
        batch_x = batch_x.to(device)
        batch_c = batch_c.to(device)
        optimizer.zero_grad()
        x_hat,mu,logvar = model(batch_x,batch_c)
        loss,recon,kl = cvae_loss(x_hat,batch_x,mu,logvar,beta = 0.01)
        loss.backward()
        optimizer.step()
        train_total_loss +=loss.item()
    avg_train_loss = train_total_loss/len(train_loader)
    ## 검증하기
    model.eval()
    val_total_loss = 0.0
    with torch.no_grad():
        for v_x,v_c in val_loader:
            v_x = v_x.to(device)
            v_c = v_c.to(device)
            v_hat,v_mu,v_logvar = model(v_x,v_c)
            v_loss,_,_ = cvae_loss(v_hat,v_x,v_mu,v_logvar,beta = 0.01)
            val_total_loss += v_loss
    avg_val_loss = val_total_loss/len(val_loader)
    train_losses.append(avg_train_loss)
    val_losses.append(avg_val_loss)
    if epoch % 20 == 0 or epoch == 1:
        print(f"Epoch [{epoch}/{epochs}] Train Loss: {avg_train_loss:.6f} | Val Loss: {avg_val_loss:.6f}")


Epoch [1/200] Train Loss: 0.043043 | Val Loss: 0.691599
Epoch [20/200] Train Loss: 0.039732 | Val Loss: 0.714723
Epoch [40/200] Train Loss: 0.037809 | Val Loss: 0.749813
Epoch [60/200] Train Loss: 0.038721 | Val Loss: 0.756095
Epoch [80/200] Train Loss: 0.034712 | Val Loss: 0.785348
Epoch [100/200] Train Loss: 0.042005 | Val Loss: 0.804880
Epoch [120/200] Train Loss: 0.031831 | Val Loss: 0.829445
Epoch [140/200] Train Loss: 0.034393 | Val Loss: 0.893272
Epoch [160/200] Train Loss: 0.031027 | Val Loss: 0.853076
Epoch [180/200] Train Loss: 0.032943 | Val Loss: 0.878430
Epoch [200/200] Train Loss: 0.040108 | Val Loss: 0.873611


In [ ]:
learning_rate = 0.01
epochs = 200
train
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)
train_total_loss = []
val_total_loss = 
for epoch in range(1,epochs+1):
    model.train()
    train_total_loss=0
    for batch_x,batch_c in train_loader:
        batch_x = batch_x.to(device)
        batch_c = batch_c.to(device)
        optimizer.zero_grad()
        x_hat,mu,logvar

        


56

In [ ]:
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)
epochs = 200

for epoch in range(1,epochs+1):
    model.train()
    train_total_loss = 0
    for batch_x,batch_c in train_loader:
        batch_x = batch_x.to(device)
        batch_c = batch_c.to(device)
        optimizer.zero_grad()
        loss,recon,kl = CVAE(batch_x,batch_c)
        loss.backward()
        optimizer.step()
        train_total_loss +=loss.item()
    avg_train_loss = train_total_loss/len(train_loader)
    model.eval()
    val_total_loss = 0
    with torch.no_grad():
        for v_x,v_c in val_loader:
            v_x = v_x.to(device)
            v_c = v_c.to(device)
            v_hat,v_mu,v_logvar = model(v_x,v_c)
            v_loss,_,_ = cvae_loss(v_hat,v_x,v_mu)
            val_total_loss += v_loss.item()
        avg_val_loss = val_total_loss/len(val_loader)